In [1]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

abstract_df = pd.read_csv('./special_abstract.csv')
abstract_specials = abstract_df['special'].values
invention_title_df = pd.read_csv('./special_invention_title.csv')
invention_title_specials = invention_title_df['special'].values
claims_df = pd.read_csv('./special_claims.csv')
claims_specials = claims_df['special'].values

In [2]:
import numpy as np

special_df = pd.DataFrame(np.concatenate((abstract_specials, invention_title_specials, claims_specials)), columns=['specials'])
special_df = special_df.drop_duplicates('specials') # 중복 제거
specials = list(special_df['specials'].values)

special_df['convert'] = np.nan

In [3]:
def find_special(df, special):
    if special not in df['special'].unique():
        return ''
    return df.iloc[df[df['special']==special].index, 1].values[0] # 1번째가 text

def show_widgets():
    """
    1.현재 special을 보여주는 항목
    
    2.이 special을 바꿀 텍스트를 입력할 항목
    
    3.입력된 텍스트를 저장하고 다음으로 넘어가기 위해 클릭할 버튼

    4.해당 special에 대한 abstract (없으면 ''로 출력)
    5.해당 special에 대한 invention_title (없으면 ''로 출력)
    6.해당 special에 대한 claims (없으면 ''로 출력)
    
    형식
    [1] [2] [3]
    abstract: [4]
    invention_titl: [5]
    claims: [6]
    """
    slider = widgets.IntSlider(min=0, max=len(specials)-1, description='slider:')

    def change_idx(change):
        idx = change['new']
        special_box.value = specials[idx]
        convert_box.value = ''
        abstract_box.value = find_special(abstract_df, specials[idx])
        invention_title_box.value = find_special(invention_title_df, specials[idx])
        claims_box.value = find_special(claims_df, specials[idx])

    slider.observe(change_idx, names='value')
    
    special_box = widgets.Text(
        value=f'{specials[0]}',
        description='특수문자:',
        disabled=True  
    )
        
    convert_box = widgets.Text(
        value='',
        placeholder='변환할 텍스트 입력',
        description='Convert:',
        disabled=False   
    )

    next_button = widgets.Button(
        description='저장 후 다음으로',
        disabled=False,
    )
    
    postpone_button = widgets.Button(
        description='보류',
        disabled=False,
    )

    def next_click_callback(clicked_button):
        special_df.loc[slider.value, 'convert'] = convert_box.value
        slider.value += 1

    next_button.on_click(next_click_callback)
    
    def postpone_click_callback(clicked_button):
        special_df.loc[slider.value, 'convert'] = '보류'
        slider.value += 1

    postpone_button.on_click(postpone_click_callback)

    hbox = widgets.HBox([special_box, convert_box, next_button, postpone_button])

    invention_title_box = widgets.HTML(
        value=find_special(invention_title_df, specials[0]),
        description='Title :',
    )
    abstract_box = widgets.HTML(
        value=find_special(abstract_df, specials[0]),
        description='Abstract :',
    )
    claims_box = widgets.HTML(
        value=find_special(claims_df, specials[0]),
        description='Claims :',
    )

    display(slider, hbox, invention_title_box, abstract_box, claims_box)

show_widgets()

IntSlider(value=0, description='slider:', max=360)

HTML(value='항균성.항곰팡이성 실리콘 고무 조성물', description='Title :')

HTML(value='이 고안은 CD가 장착된 녹음장치에 관한 것으로서, 더욱 상세하게는 CD가 수록된 곡을 테이프에 녹음할 때에 녹음기를 누름과 동시에 콘덴서에 충전시켜 CD의 동작시간을 지연시킨…

HTML(value='카세트 테이프 플레이어와 CD 플레이어가 일체화된 장치에서, 녹음키가 입력되면 CD 플레이어가 재생동작을 함과 동시에, 재생된 신호가 카세트 테이프 플레이어에 녹음되도록 하기 …

In [13]:
print(special_df.info())
special_df

<class 'pandas.core.frame.DataFrame'>
Index: 379 entries, 0 to 360
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   specials  361 non-null    object
 1   convert   361 non-null    object
dtypes: object(2)
memory usage: 17.0+ KB
None


,specials,convert
0,.,.
1,",",","
2,(,보류
3,→,","
4,),보류
...,...,...
356,NaN,w
357,NaN,M
358,NaN,N
359,NaN,B


In [14]:
special_df.to_csv('./special_to_normal.csv', index=False)

# 보류한 특수 문자 확인

In [13]:
special_df = pd.read_csv('./special_to_normal_first.csv')
train_df = pd.read_csv('./baseline/train.csv')
train_df

,documentId,invention_title,abstract,claims,SSnos
0,kr19910011048y1,콤팩트 디스크를 동기 녹음회로,"이 고안은 CD가 장착된 녹음장치에 관한 것으로서, 더욱 상세하게는 CD가 수록된 ...","카세트 테이프 플레이어와 CD 플레이어가 일체화된 장치에서, 녹음키가 입력되면 CD...",26521
1,kr19910015448a,녹음 테이프의 비정상 더빙 부위 소거방법,"본 발명은 테이프음의 더빙에 관한 것으로, 일반적으로 사용되고 있는 더빙 방식은 녹...",재생 테이프의 곡을 녹음 테이프로 더빙시 재생 테이프가 먼저 끝나면 더빙 과정을 종...,59120
2,kr19910017768a,브이씨알의 영상 더빙 자동 기록 제어장치 및 방법,본 발명의 브이씨알의 영상 더빙 자동 기록 제어장치 및 방법은 재생되는 영상신호의 ...,마이콤(1)의 출력단자(OT)와 앤드게이트(2)의 출력단자를 오아게이트(8)를 통해...,59120
3,kr19910022167b1,영상기록재생장치의 더빙회로,영상기록재생장치의 더빙회로는 한 채널의 영상신호를 소거하여 다른 원하는 신호를 기록...,"두 개의 기록재생헤드와 두 개의 소거헤드를 구비한 영상기록 재생장치에 있어서, 기록...",59120
4,kr19910023691a,캠코더의 오디오 더빙회로,"본 발명은 8MM 캠코더의 더빙 및 재생 제어회로에 관한 것으로, 현재까지 일반 가...","외부마이크로(MIC1)의 신호나 라인 음성신호를 FM 변조처리함과 아울러, 재생신호...",59120
...,...,...,...,...,...
59995,kr20217028291a,가금류의 장각부를 발골하기 위한 시스템 및 방법,"적어도 하나의 뼈 및 상기 뼈를 둘러싸는 살코기를 포함하는, 가금류의 장각부(leg...","적어도 하나의 뼈 및 상기 뼈를 둘러싸는 살코기를 포함하는, 가금류의 장각부(leg...",10121
59996,kr20217028452a,도금 강판,"이 도금 강판은, 강판과, 상기 강판의 표면의 적어도 일부에 형성된 도금층을 갖고,...","강판과,상기 강판의 표면의 적어도 일부에 형성된 도금층을 갖고,상기 도금층의 화학 ...",24191
59997,kr20217028893a,"전처리용 혼합 교반기, 석고 슬러리 제조 장치, 건축용 면재 제조 장치, 전처리 소...",소석고와 물을 혼련하여 석고 슬러리로 하는 슬러리 형성용 혼합 교반기의 전단에 배치...,소석고와 물을 혼련하여 석고 슬러리로 하는 슬러리 형성용 혼합 교반기의 전단에 배치...,23323
59998,kr20217029041a,플루오로락톤 및 그 제조 방법,"본 개시는, 헥사플루오로프로필렌옥시드 등으로부터 1 공정에서 플루오로락톤 화합물을 ...","식 (1):[식 중, 2개의 R1은 동일하고, 불소 원자 또는 플루오로알킬기이다.]...",20119


In [19]:
special_df

,specials,convert
0,.,.
1,",",","
2,(,(
3,→,","
4,),)
...,...,...
356,ｗ,w
357,Μ,M
358,Ν,N
359,Β,B


In [5]:
import pandas as pd
pd.set_option("display.max_rows", 370)

In [8]:
df = pd.read_csv('./special_to_normal_first.csv')
df[df['convert']=='보류'].reset_index()

,index,specials,convert
0,8,－,보류
1,17,-,보류
2,19,&,보류
3,21,+,보류
4,22,δ,보류
5,23,×,보류
6,32,=,보류
7,41,〈,보류
8,43,」,보류
9,46,「,보류


In [11]:
df.loc[151, 'specials']

'\xad'

In [16]:
for i, text in train_df['abstract'].items():
    if '\xad' in text:
        print(text)
        break

본 발명은 자동차의 연료 및 유압배관에 사용되는 수지 피복 알루미늄 튜브및 그 제조방법에 관한 것으로, 특히, 알루미늄 튜브의 표면에 폴리머 수지를 피복하여 이루어지는 자동차 배관용 수지 피복 알루미늄 튜브 및 그 제조방법에 관한 것이다. 자동차 배관용 수지 피복 알루미늄 튜브의 제조방법은 압출­인발에 의해 제조된 알루미늄 튜브를 공급하는 공급단계와; 알루미늄 튜브의 표면을 성막하는 표면성막단계와; 알루미늄 튜브를 예열하는 예열단계와; 알루미늄 튜브에 공압출(Co-Extrusion)로 폴리머 수지를 피복하는 피복단계를 포함하여 이루어지는 것을 특징으로 한다.


In [19]:
train_df.loc[3337,'abstract']

'본 발명은 자동차의 연료 및 유압배관에 사용되는 수지 피복 알루미늄 튜브및 그 제조방법에 관한 것으로, 특히, 알루미늄 튜브의 표면에 폴리머 수지를 피복하여 이루어지는 자동차 배관용 수지 피복 알루미늄 튜브 및 그 제조방법에 관한 것이다. 자동차 배관용 수지 피복 알루미늄 튜브의 제조방법은 압출\xad인발에 의해 제조된 알루미늄 튜브를 공급하는 공급단계와; 알루미늄 튜브의 표면을 성막하는 표면성막단계와; 알루미늄 튜브를 예열하는 예열단계와; 알루미늄 튜브에 공압출(Co-Extrusion)로 폴리머 수지를 피복하는 피복단계를 포함하여 이루어지는 것을 특징으로 한다.'